# 1. Import Libraries

In [ ]:
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29

In [3]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

In [4]:
from crewai import Agent, Task, Crew

- LLM model: HuggingFace model

In [40]:
from langchain_community.llms import HuggingFaceHub

llm = HuggingFaceHub(
    repo_id="mistralai/Mistral-7B-Instruct-v0.1",
    huggingfacehub_api_token="HF_TOKEN",
    task="text-generation",
    model_kwargs={
        "temperature": 0.7,
        "max_new_tokens": 500
    }
)

## 2- Creating Agents

- Define your Agents, and provide them a `role`, `goal` and `backstory`.
- When role playing for agent is defined, model performance increases.

## Agent-1 : Planner

In [41]:
planner = Agent(
    role="Content Planner",
    goal="Plan engaging and factually accurate content on {topic}",
    backstory="You're working on planning a blog article "
              "about the topic: {topic}."
              "You collect information that helps the "
              "audience learn something "
              "and make informed decisions. "
              "Your work is the basis for "
              "the Content Writer to write an article on this topic.",
    llm=llm,
    allow_delegation=False,
	verbose=True
)

### Agent-2: Writer

In [42]:
writer = Agent(
    role="Content Writer",
    goal="Write insightful and factually accurate "
         "opinion piece about the topic: {topic}",
    backstory="You're working on a writing "
              "a new opinion piece about the topic: {topic}. "
              "You base your writing on the work of "
              "the Content Planner, who provides an outline "
              "and relevant context about the topic. "
              "You follow the main objectives and "
              "direction of the outline, "
              "as provide by the Content Planner. "
              "You also provide objective and impartial insights "
              "and back them up with information "
              "provide by the Content Planner. "
              "You acknowledge in your opinion piece "
              "when your statements are opinions "
              "as opposed to objective statements.",
    llm=llm,
    allow_delegation=False,
    verbose=True
)

### Agent-3: Editor

In [43]:
editor = Agent(
    role="Editor",
    goal="Edit a given blog post to align with "
         "the writing style of the organization. ",
    backstory="You are an editor who receives a blog post "
              "from the Content Writer. "
              "Your goal is to review the blog post "
              "to ensure that it follows journalistic best practices,"
              "provides balanced viewpoints "
              "when providing opinions or assertions, "
              "and also avoids major controversial topics "
              "or opinions when possible.",
    llm=llm,
    allow_delegation=False,
    verbose=True
)

## Creating Tasks
- Provide them a `description`, `expected_output` and `agent`.

### Task-1: Plan

In [44]:
plan = Task(
    description=(
        "1. Prioritize the latest trends, key players, "
            "and noteworthy news on {topic}.\n"
        "2. Identify the target audience, considering "
            "their interests and pain points.\n"
        "3. Develop a detailed content outline including "
            "an introduction, key points, and a call to action.\n"
        "4. Include SEO keywords and relevant data or sources."
    ),
    expected_output="A comprehensive content plan document "
        "with an outline, audience analysis, "
        "SEO keywords, and resources.",
    agent=planner,
)

### Task-2: Write

In [45]:
write = Task(
    description=(
        "1. Use the content plan to craft a compelling "
            "blog post on {topic}.\n"
        "2. Incorporate SEO keywords naturally.\n"
		"3. Sections/Subtitles are properly named "
            "in an engaging manner.\n"
        "4. Ensure the post is structured with an "
            "engaging introduction, insightful body, "
            "and a summarizing conclusion.\n"
        "5. Proofread for grammatical errors and "
            "alignment with the brand's voice.\n"
    ),
    expected_output="A well-written blog post "
        "in markdown format, ready for publication, "
        "each section should have 2 or 3 paragraphs.",
    agent=writer,
)

### Task-3: Edit

In [46]:
edit = Task(
    description=("Proofread the given blog post for "
                 "grammatical errors and "
                 "alignment with the brand's voice."),
    expected_output="A well-written blog post in markdown format, "
                    "ready for publication, "
                    "each section should have 2 or 3 paragraphs.",
    agent=editor
)

## Creating the Crew
- `verbose=2` represents detail level of execution.

In [47]:
crew = Crew(
    agents=[planner, writer, editor],
    tasks=[plan, write, edit],
    verbose=2
)

## Running the Crew

In [48]:
result = crew.kickoff(inputs={"topic": "Artificial Intelligence"})

 [DEBUG]: == Working Agent: Content Planner
 [INFO]: == Starting Task: 1. Prioritize the latest trends, key players, and noteworthy news on Artificial Intelligence.
2. Identify the target audience, considering their interests and pain points.
3. Develop a detailed content outline including an introduction, key points, and a call to action.
4. Include SEO keywords and relevant data or sources.


> Entering new CrewAgentExecutor chain...
You are Content Planner. You're working on planning a blog article about the topic: Artificial Intelligence.You collect information that helps the audience learn something and make informed decisions. Your work is the basis for the Content Writer to write an article on this topic.
Your personal goal is: Plan engaging and factually accurate content on Artificial IntelligenceTo give my best complete final answer to the task use the exact following format:

Thought: I now can give a great answer
Final Answer: my best complete final answer to the task.
Your 

In [49]:
# Show texts properly
from IPython.display import Markdown
Markdown(result)

Artificial Intelligence: The Latest Trends, Key Players, and Benefits
=================================================================

As technology continues to advance, artificial intelligence (AI) is becoming an increasingly important part of our daily lives. AI refers to the development of intelligent machines that can perform tasks that typically require human intelligence, such as understanding natural language and recognizing images.

One of the main benefits of AI is its ability to improve efficiency and productivity in various industries. For example, in healthcare, AI is being used to analyze medical images and help doctors make more accurate diagnoses. In finance, AI is being used to predict market trends and assist in risk management. In transportation, AI is being used to develop self-driving cars and improve traffic flow.

Google is one of the key players in the AI industry, having invested heavily in AI research and development. Other major companies in the field include Amazon, Microsoft, and IBM, which are also developing products and services that leverage AI to provide solutions to a wide range of problems.

Recent advancements in deep learning algorithms have allowed for more accurate object recognition in images, which has the potential to revolutionize the way we interact with technology and improve our daily lives. However, it is important to consider the ethical implications of AI as it becomes more integrated into our daily lives. Issues such as privacy and bias must be addressed in order to ensure that AI is used in a responsible and ethical manner.

If you are interested in learning more about AI and its potential benefits, there are several resources available. TechCrunch provides the latest trends in AI research and development, while Harvard Business Review explores the ethical implications of AI.

In conclusion, AI is a rapidly growing field that has the potential to revolutionize the way we live and work. As new products and services are developed and advancements are made in AI, it is important to ensure that it is used in a responsible and ethical manner.

In [50]:
# Result for Second Topic
topic = "How does the integration of artificial intelligence technologies in the U.S. technology sector affect employment rates and wage inequality between high-skilled and low-skilled workers?"
result = crew.kickoff(inputs={"topic": topic})

 [DEBUG]: == Working Agent: Content Planner
 [INFO]: == Starting Task: 1. Prioritize the latest trends, key players, and noteworthy news on How does the integration of artificial intelligence technologies in the U.S. technology sector affect employment rates and wage inequality between high-skilled and low-skilled workers?.
2. Identify the target audience, considering their interests and pain points.
3. Develop a detailed content outline including an introduction, key points, and a call to action.
4. Include SEO keywords and relevant data or sources.


> Entering new CrewAgentExecutor chain...
You are Content Planner. You're working on planning a blog article about the topic: How does the integration of artificial intelligence technologies in the U.S. technology sector affect employment rates and wage inequality between high-skilled and low-skilled workers?.You collect information that helps the audience learn something and make informed decisions. Your work is the basis for the Conten

In [51]:
Markdown(result)

Here's a final version of the blog post with some minor edits for grammar and clarity:

The integration of artificial intelligence (AI) technologies in the U.S. technology sector has the potential to significantly affect employment rates and wage inequality between high-skilled and low-skilled workers. As AI becomes more advanced, it is increasingly being used to perform tasks that were previously done by human workers, such as automation in the workplace. This has led to concerns about the impact of automation on employment rates and wage inequality, as low-skilled workers may be the most vulnerable to job loss.

Additionally, the rise of the gig economy is another key player to consider. Many gig workers are already facing wage inequality, and the integration of AI into hiring processes could make this even more pronounced. Additionally, the use of AI in hiring could make it more difficult for workers to find stable employment, as the algorithms used by these systems may be biased against certain groups.

To address these concerns, it is important to consider the potential benefits of AI integration into the workplace. For example, AI can help to improve productivity and efficiency, which could ultimately lead to better wages for all workers. Additionally, AI can help to identify and address bias in hiring processes, which could help to reduce wage inequality.

However, it is important to note that the impact of AI on employment rates and wage inequality will depend on a variety of factors, including the specific industries and jobs affected, the speed of technological advancements, and the ability of workers to adapt to new technologies. Additionally, the increasing use of AI in hiring processes could lead to a greater reliance on algorithms, which could make it more difficult for workers to find stable employment.

In conclusion, the integration of artificial intelligence technologies in the U.S. technology sector has the potential to significantly affect employment rates and wage inequality between high-skilled and low-skilled workers. It is important for polic

<a name='1'></a>
 ## Other Options Models as LLM for Agents

### Gpt-3.5-turbo Model

In [ ]:
"""
import os
from utils import get_openai_api_key

openai_api_key = get_openai_api_key()
os.environ["OPENAI_MODEL_NAME"] = 'gpt-3.5-turbo'
"""

#### Hugging Face (HuggingFaceHub endpoint)

```Python
from langchain_community.llms import HuggingFaceHub

llm = HuggingFaceHub(
    repo_id="HuggingFaceH4/zephyr-7b-beta",
    huggingfacehub_api_token="<HF_TOKEN_HERE>",
    task="text-generation",
)

### you will pass "llm" to your agent function
```

#### Mistral API

```Python
OPENAI_API_KEY=your-mistral-api-key
OPENAI_API_BASE=https://api.mistral.ai/v1
OPENAI_MODEL_NAME="mistral-small"
```

#### Cohere

```Python
from langchain_community.chat_models import ChatCohere
# Initialize language model
os.environ["COHERE_API_KEY"] = "your-cohere-api-key"
llm = ChatCohere()

### you will pass "llm" to your agent function
```